# Imports

In [ ]:
# ruff: noqa
from pydantic import BaseModel, Field


from langsmith import traceable

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

from langchain_core.messages import convert_to_openai_messages

from jinja2 import Template
from typing import Dict, Any, Annotated, List
from operator import add
from openai import OpenAI


import instructor

from utils.utils import get_tool_descriptions, format_ai_message
from utils.tools import (
    get_formatted_context,
    get_formatted_reviews_context,
    get_shopping_cart,
    remove_from_cart,
    add_to_shopping_cart,
)

from langgraph.checkpoint.postgres import PostgresSaver

# Create Coordinator Dataset

In [ ]:
class Delegation(BaseModel):
    agent: str
    task: str


class CoordinatorAgentResponse(BaseModel):
    next_agent: str
    plan: List[Delegation]
    answer: str
    final_answer: bool

In [ ]:
from langchain_core.messages import AIMessage


@traceable(
    name="coordinator_agent",
    run_type="llm",
    metadata={"ls_provider": "openai", "ls_model_name": "gpt-4.1"},
)
def coordinator_agent(state: "State"):
    prompt_template = """You are a Coordinator Agent as part of a shopping assistant.

Your role is to create plans for solving user queries and delegate the tasks accordingly.
You will be given a conversation history, your task is to create a plan for solving the user's query.
After the plan is created, you should output the next agent to invoke and the task to be performed by that agent.
Once an agent finishes its task, you will be handed the control back, you should then review the conversation history and revise the plan.
If there is a sequence of tasks to be performed by a single agent, you should combine them into a single task.

The possible agents are:

- product_qa_agent: The user is asking a question about a product. This can be a question about available products, their specifications, user reviews etc.
- shopping_cart_agent: The user is asking to add or remove items from the shopping cart or questions about the current shopping cart.

CRITICAL RULES:
- If next_agent is "", final_answer MUST be false
(You cannot delegate the task to an agent and return to the user in the same response)
- If final_answer is true, next_agent MUST be ""
(You must wait for agent results before returning to user)
- If you need to call other agents before answering, set:
next_agent="...", final_answer=false
- After receiving agent results, you can then set:
next_agent="", final_answer=true
- One of the following has to be true:
next_agent is "" and final_answer is true
next_agent is not "" and final_answer is false

Additional instructions:

- Do not route to any agent if the user's query needs clarification. Do it yourself.
- Write the plan to the plan field.
- Write the next agent to invoke to the next_agent field.
- Once you have all the information needed to answer the user's query, you should set the final_answer field to True and output the answer to the user's query.
- The final answer to the user query should be a comprehensive answer that explains the actions that were performed to answer the query.
- Never set final_answer to true if the plan is not complete.
- You should output the next_agent field as well as the plan field.
"""

    template = Template(prompt_template)

    prompt = template.render()

    messages = state.messages

    conversation = []

    for message in messages:
        conversation.append(convert_to_openai_messages(message))

    client = instructor.from_openai(OpenAI())

    response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1",
        response_model=CoordinatorAgentResponse,
        messages=[{"role": "system", "content": prompt}, *conversation],
        temperature=0.0,
    )

    if response.final_answer:
        ai_message = []
    else:
        ai_message = [AIMessage(content=response.answer)]

    return {
        "messages": ai_message,
        "answer": response.answer,
        "coordinator_agent": {
            "iteration": state.coordinator_agent.iteration + 1,
            "final_answer": response.final_answer,
            "next_agent": response.next_agent,
            "plan": [data.model_dump() for data in response.plan],
        },
    }

In [ ]:
class ToolCall(BaseModel):
    name: str
    arguments: dict


class RAGUsedContext(BaseModel):
    id: str = Field(description="ID of the item used to answer the question")
    description: str = Field(description="Description of the item used to answer")


class AgentProperties(BaseModel):
    iteration: int = 0
    final_answer: bool = False
    available_tools: List[Dict[str, Any]] = []
    tool_calls: List[ToolCall] = []


class CoordinatorAgentProperties(BaseModel):
    iteration: int = 0
    final_answer: bool = False
    plan: List[Delegation] = []
    next_agent: str = ""


class State(BaseModel):
    messages: Annotated[List[Any], add] = []
    user_intent: str = ""
    product_qa_agent: AgentProperties = Field(default_factory=AgentProperties)
    shopping_cart_agent: AgentProperties = Field(default_factory=AgentProperties)
    coordinator_agent: CoordinatorAgentProperties = Field(
        default_factory=CoordinatorAgentProperties
    )
    answer: str = ""
    references: Annotated[List[RAGUsedContext], add] = []
    user_id: str = ""
    cart_id: str = ""

In [ ]:
initial_state = State(
    messages=[{"role": "user", "content": "How is the weather today?"}]
)

In [ ]:
coordinator_agent(initial_state)

In [ ]:
coordinator_eval_dataset = [
    {
        "inputs": {
            "messages": [{"role": "user", "content": "What is the weather today?"}]
        },
        "outputs": {"next_agent": "", "coordinator_final_answer": True},
    },
    {
        "inputs": {
            "messages": [{"role": "user", "content": "Can I get some earphones?"}]
        },
        "outputs": {
            "next_agent": "product_qa_agent",
            "coordinator_final_answer": False,
        },
    },
    {
        "inputs": {
            "messages": [
                {
                    "role": "user",
                    "content": "Can you add an item with ID B09NLTDHQ6 to my cart?",
                }
            ]
        },
        "outputs": {
            "next_agent": "shopping_cart_agent",
            "coordinator_final_answer": False,
        },
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you add those earphones to my cart?"}
            ]
        },
        "outputs": {"next_agent": "", "coordinator_final_answer": True},
    },
    {
        "inputs": {
            "messages": [
                {
                    "role": "user",
                    "content": "Can you add the best items to my cart? I am looking for laptop bags.",
                }
            ]
        },
        "outputs": {
            "next_agent": "product_qa_agent",
            "coordinator_final_answer": False,
        },
    },
    {
        "inputs": {
            "messages": [
                {
                    "role": "user",
                    "content": "Can you find some good reviews for items in my cart?",
                }
            ]
        },
        "outputs": {
            "next_agent": "shopping_cart_agent",
            "coordinator_final_answer": False,
        },
    },
    {
        "inputs": {
            "messages": [
                {
                    "role": "user",
                    "content": "Can you put the items with the most positive user reviews to my cart?",
                }
            ]
        },
        "outputs": {
            "next_agent": "product_qa_agent",
            "coordinator_final_answer": False,
        },
    },
    {
        "inputs": {
            "messages": [{"role": "user", "content": "What kind of stuff do you sell?"}]
        },
        "outputs": {"next_agent": "", "coordinator_final_answer": True},
    },
    {
        "inputs": {
            "messages": [{"role": "user", "content": "Can you help me with my order?"}]
        },
        "outputs": {"next_agent": "", "coordinator_final_answer": True},
    },
    {
        "inputs": {
            "messages": [
                {
                    "role": "user",
                    "content": "Can you add two, ideally red tablets to my cart?",
                }
            ]
        },
        "outputs": {
            "next_agent": "product_qa_agent",
            "coordinator_final_answer": False,
        },
    },
]

# Upload dataset to Langsmith

In [ ]:
from langsmith import Client
import os

client = Client(api_key=os.environ["LANGSMITH_API_KEY"])
dataset_name = "coordinator-eval-dataset"

# dataset = client.create_dataset(
#     dataset_name=dataset_name,
#     description="Dataset for evaluating routing of the coordinator agent"
# )
dataset = client.share_dataset(dataset_name=dataset_name)

In [ ]:
for item in coordinator_eval_dataset:
    client.create_example(
        dataset_id=dataset.id,
        inputs={"messages": item["inputs"]["messages"]},
        outputs={
            "next_agent": item["outputs"]["next_agent"],
            "coordinator_final_answer": item["outputs"]["coordinator_final_answer"],
        },
    )

In [ ]:
def next_agent_evaluator(run, example):
    print(run.outputs)
    next_agent_match = (
        run.outputs["coordinator_agent"]["next_agent"] == example.outputs["next_agent"]
    )
    final_answer_match = (
        run.outputs["coordinator_agent"]["final_answer"]
        == example.outputs["coordinator_final_answer"]
    )
    return next_agent_match and final_answer_match

In [ ]:
results = client.evaluate(
    lambda x: coordinator_agent(State(messages=x["messages"])),
    data=dataset_name,
    evaluators=[next_agent_evaluator],
    experiment_prefix="coordinator-eval-dataset",
)

In [ ]:
results